In [1]:
# load Preliminaries
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random
import networkx as nx

In [64]:
###### Parameters #######
DATASETS = ['letter', 'AIDS', 'mutagenicity']
dataset = DATASETS[0]

In [65]:
# loading the engines locally along with plug-in.
HTML('''
<script src="./sigma.min.js"></script>
<script src="./sigma-add-method-neighbors.js"></script>

''')

#<style>
      #graph-div {
 #       width: 800px;
  #      height: 800px;
   #     -webkit-transform: rotate(180deg);
    #    -moz-transform: rotate(180deg);
     #   -o-transform: rotate(180deg);
      #  transform: rotate(180deg);
      #}
    #</style>

In [66]:
filename = '../results/draw/letter/AP1_0000.json'
# filename = '../results/draw/AIDS/molid2184.json'
# filename = '../results/draw/mutagenicity/molecule_1.json'
with open(filename, mode='r') as fp:
    data = json.load(fp) #.readlines()
#data = json.load()
print(data)

{'nodes': [{'id': '0', 'label': 'PageRank: 0.154; x:0.687437, y:0.271509', 'x': 0.687, 'y': 0.272, 'size': 0.154}, {'id': '1', 'label': 'PageRank: 0.292; x:1.39826, y:1.84766', 'x': 1.398, 'y': 1.848, 'size': 0.292}, {'id': '2', 'label': 'PageRank: 0.154; x:3.14525, y:0.904519', 'x': 3.145, 'y': 0.905, 'size': 0.154}, {'id': '3', 'label': 'PageRank: 0.2; x:0.93198, y:1.37192', 'x': 0.932, 'y': 1.372, 'size': 0.2}, {'id': '4', 'label': 'PageRank: 0.2; x:2.00406, y:1.48344', 'x': 2.004, 'y': 1.483, 'size': 0.2}], 'edges': [{'id': '0', 'source': '0', 'target': '1', 'label': '0'}, {'id': '1', 'source': '1', 'target': '2', 'label': '0'}, {'id': '2', 'source': '2', 'target': '1', 'label': '0'}, {'id': '3', 'source': '3', 'target': '4', 'label': '0'}, {'id': '4', 'source': '4', 'target': '3', 'label': '0'}, {'id': '5', 'source': '1', 'target': '0', 'label': '0'}]}


In [67]:
graph_data = data

In [73]:
if dataset == 'mutagenicity':
    nx_graph = nx.Graph()
    for node in graph_data['nodes']:
        nx_graph.add_node(node['id'])
    
    for edge in graph_data['edges']:
        nx_graph.add_edge(edge['source'], edge['target'], weight=int(edge['label']))
    
    # pos = nx.drawing.layout.spring_layout(nx_graph, iterations=2500, center=[0.0, 0.0], seed=42)
    pos = nx.drawing.layout.kamada_kawai_layout(nx_graph)

    for node in graph_data['nodes']:
        node['x'], node['y'] = pos[node['id']]

In [69]:
js_text_template = Template('''
    
    var g = $graph_data ;

s = new sigma({graph: g, container: '$container', settings: { defaultNodeColor: '#ec5148', labelThreshold: 6, scalingMode: 'inside'} });

s.graph.nodes().forEach(function(n) {
  n.originalColor = n.color;
});
s.graph.edges().forEach(function(e) {
  e.originalColor = e.color;
});

s.bind('clickNode', function(e) {
  var nodeId = e.data.node.id,
      toKeep = s.graph.neighbors(nodeId);
  toKeep[nodeId] = e.data.node;

  s.graph.nodes().forEach(function(n) {
    if (toKeep[n.id])
      n.color = n.originalColor;
    else
      n.color = '#eee';
  });

  s.graph.edges().forEach(function(e) {
    if (toKeep[e.source] && toKeep[e.target])
      e.color = e.originalColor;
    else
      e.color = '#eee';
  });

  s.refresh();
});

s.bind('clickStage', function(e) {
  s.graph.nodes().forEach(function(n) {
    n.color = n.originalColor;
  });

  s.graph.edges().forEach(function(e) {
    e.color = e.originalColor;
  });

  s.refresh();
});


''')

In [70]:
js_text = js_text_template.substitute({'graph_data': json.dumps(graph_data),
                                       'container': 'graph-div'})


In [71]:
html_template = Template('''
<div id="graph-div" style="height:800px"></div>
<script> $js_text </script>
''')

In [72]:
HTML(html_template.substitute({'js_text': js_text}))